In [59]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


In [60]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "mgmt-467-2500"   # <-- EDIT ME
# assert PROJECT_ID != "mgmt-467-2500", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
client = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: mgmt-467-2500


In [61]:
DATASET = "Assignment_1"
TABLE   = "inventory_items"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = client.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,127865,13844,2025-07-14 08:52:54+00:00,2025-08-06 04:28:54+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,127866,13844,2025-02-26 12:06:00+00:00,NaT,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,293178,13844,2023-07-23 22:35:58+00:00,2023-09-17 04:55:58+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,293179,13844,2025-01-13 12:41:00+00:00,NaT,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,293180,13844,2025-02-06 15:21:00+00:00,NaT,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


✅ Connection OK.


# 1. Discover

### KPI 1: Monthly Revenue

BigQuery SQL Only

**"Generate an SQL query that calculates the monthly revenue and its Month-over-Month (MoM) and Year-over-Year (YoY) growth percentages.

Use CTEs and window functions.

The price is the sale_price from order_items, falling back to retail_price from products.

Use these tables:

`mgmt-467-2500.Assignment_1.order_items` (oi)

`mgmt-467-2500.Assignment_1.products` (p)

`mgmt-467-2500.Assignment_1.orders` (o)

Order the result by month descending and limit it to 10 rows."**

In [62]:
query_string = """
-- Monthly revenue with MoM and YoY growth (CTE + windows)
WITH item_price AS (
  SELECT
    oi.order_id,
    oi.product_id,
    COALESCE(oi.sale_price, p.retail_price) AS item_price,
    oi.* EXCEPT(order_id, product_id)  -- keep other fields if needed
  FROM `mgmt-467-2500.Assignment_1.order_items` oi
  LEFT JOIN `mgmt-467-2500.Assignment_1.products` p
    ON oi.product_id = p.id
),
monthly_revenue AS (
  SELECT
    DATE_TRUNC(DATE(o.created_at), MONTH) AS month,
    SUM(ip.item_price) AS revenue
  FROM `mgmt-467-2500.Assignment_1.orders` o
  JOIN item_price ip USING(order_id)
  GROUP BY month
),
trend AS (
  SELECT
    month,
    revenue,
    LAG(revenue) OVER (ORDER BY month) AS revenue_prev_month,
    LAG(revenue, 12) OVER (ORDER BY month) AS revenue_prev_year
  FROM monthly_revenue
)
SELECT
  month,
  revenue,
  SAFE_DIVIDE(revenue - revenue_prev_month, revenue_prev_month) AS mom_change_pct,
  SAFE_DIVIDE(revenue - revenue_prev_year, revenue_prev_year) AS yoy_change_pct
FROM trend
ORDER BY month DESC
LIMIT 10;


"""
query_job = client.query(query_string)

results_df_mr = query_job.to_dataframe()
display(results_df_mr)

,month,revenue,mom_change_pct,yoy_change_pct
0,2025-10-01,237060.060225,-0.685321,-0.079503
1,2025-09-01,753338.890734,0.393832,2.208582
2,2025-08-01,540480.560428,0.204861,1.358360
3,2025-07-01,448583.400263,0.148066,0.922670
4,2025-06-01,390729.550540,0.031168,0.929674
5,2025-05-01,378919.490309,0.091287,0.800657
6,2025-04-01,347222.740374,0.028595,0.728811
7,2025-03-01,337569.790313,0.202489,0.692366
8,2025-02-01,280726.000358,-0.012975,0.721541
9,2025-01-01,284416.400309,-0.019103,0.493712


### KPI 2: Average Order Value

#####
Write a SQL query in BigQuery syntax to calculate the average order value (AOV) for all orders. Use the following tables: order_items with columns like order_id, product_id, and sale_price. products with columns like id and retail_price. orders with columns like order_id, user_id, order_date, and gender. The query should: Join order_items with products using product_id. Use COALESCE(sale_price, retail_price) as the item price. Sum the total item prices per order to get each order’s total value. Then calculate the overall average order value (AOV) as the mean of all order totals. Include a version grouped by month (FORMAT_DATE('%Y-%m', order_date)) as well. Use table references like: mgmt-467-2500.Assignment_1.order_items mgmt-467-2500.Assignment_1.products mgmt-467-2500.Assignment_1.orders

In [63]:
query_string = """
WITH OrderTotals AS (
  SELECT
    oi.order_id,
    SUM(COALESCE(oi.sale_price, p.retail_price)) AS order_total,
    DATE(o.created_at) as order_date
  FROM `mgmt-467-2500.Assignment_1.order_items` oi
  JOIN `mgmt-467-2500.Assignment_1.orders` o ON oi.order_id = o.order_id
  LEFT JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
  GROUP BY oi.order_id, order_date
)

-- Monthly AOV
SELECT
  FORMAT_DATE('%Y-%m', order_date) AS month,
  AVG(order_total) AS monthly_aov
FROM OrderTotals
GROUP BY month
ORDER BY month;
"""
query_job = client.query(query_string)

results_df_aov = query_job.to_dataframe()
display(results_df_aov)

,month,monthly_aov
0,2019-01,124.256667
1,2019-02,63.000000
2,2019-03,90.507142
3,2019-04,108.515469
4,2019-05,84.609222
...,...,...
77,2025-06,86.120686
78,2025-07,86.034407
79,2025-08,88.184135
80,2025-09,87.010729


### KPI 3: Gross Profit Margin

##### Write a SQL query in BigQuery syntax to calculate the gross profit margin for all products and by month. Use the following tables: order_items with columns like order_id, product_id, and sale_price. products with columns like id, retail_price, cost, and category. orders with columns like order_id, order_date, and gender. The query should: Join order_items with products on product_id. Use COALESCE(sale_price, retail_price) as the item price. Calculate revenue as the sum of item prices. Calculate cost of goods sold (COGS) as the sum of product cost. Compute gross profit = revenue - COGS. Compute gross profit margin = gross profit / revenue. Group results by month using FORMAT_DATE('%Y-%m', order_date). Optionally, include filters such as p.category = 'Swim' or o.gender = 'F'. Use fully qualified table references like: mgmt-467-2500.Assignment_1.order_items mgmt-467-2500.Assignment_1.products mgmt-467-2500.Assignment_1.orders Order results by month ascending

In [64]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE(o.created_at)) AS month,
  SUM(COALESCE(oi.sale_price, p.retail_price)) AS revenue,
  SUM(p.cost) AS cogs,
  SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost) AS gross_profit,
  SAFE_DIVIDE(SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost), SUM(COALESCE(oi.sale_price, p.retail_price))) AS gross_profit_margin
FROM `mgmt-467-2500.Assignment_1.order_items` oi
JOIN `mgmt-467-2500.Assignment_1.orders` o ON oi.order_id = o.order_id
JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
GROUP BY month
ORDER BY month ASC;
"""
query_job = client.query(query_string)

results_df = query_job.to_dataframe()
display(results_df)

,month,revenue,cogs,gross_profit,gross_profit_margin
0,2019-01,1118.310001,535.711001,582.599001,0.520964
1,2019-02,1260.000001,586.530211,673.469790,0.534500
2,2019-03,3801.299984,1749.347874,2051.952110,0.539803
3,2019-04,6944.990016,3223.871488,3721.118528,0.535799
4,2019-05,7614.830006,3703.930334,3910.899672,0.513590
...,...,...,...,...,...
77,2025-06,390729.550540,187759.426834,202970.123706,0.519464
78,2025-07,448583.400263,215539.238012,233044.162252,0.519511
79,2025-08,540480.560428,260270.091934,280210.468494,0.518447
80,2025-09,753338.890734,360735.694917,392603.195816,0.521151


# Investigate

## Deep Dive into Swim Category

Prompt:
Write a SQL query that finds the top 10 products in the "Swim" category by total revenue, grouped by month.

Use two tables:

order_items with columns like product_id, sale_price, and order_date.

products with columns like id, name, retail_price, cost, and category.

The query should:

Join order_items and products on product ID.

Calculate total units sold (COUNT(1)) for each product per month.

Calculate total revenue as the sum of sale_price (or retail_price if sale_price is NULL).

Calculate total margin as revenue minus cost.

Extract the month from order_date (use FORMAT_DATE('%Y-%m', order_date) or equivalent).

Filter to only products in the Swim category.

Group by product ID, name, cost, and month.

Order by revenue in descending order.

Limit the result to the top 10 products by revenue per month.

Use BigQuery syntax and table references like mgmt-467-2500.Assignment_1.order_items.

In [65]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE(o.created_at)) AS month,
  p.id AS product_id,
  p.name,
  COUNT(1) AS units_sold,
  SUM(COALESCE(oi.sale_price, p.retail_price)) AS revenue,
  SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost) AS margin_total
FROM `mgmt-467-2500.Assignment_1.order_items` oi
JOIN `mgmt-467-2500.Assignment_1.orders` o ON oi.order_id = o.order_id
LEFT JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
WHERE p.category = 'Swim'
GROUP BY 1, p.id, p.name, p.sku, p.cost
ORDER BY revenue DESC
LIMIT 10;
"""
query_job = client.query(query_string)

results_df = query_job.to_dataframe()
display(results_df)

,month,product_id,name,units_sold,revenue,margin_total
0,2024-08,28112,Zoot Sports Men's Ultra Speedzoot Swim Race Suit,2,549.979980,217.242092
1,2024-06,13127,MiraclesuitÂ® Rialto One-Piece Swimsuit,3,523.230011,318.647075
2,2025-08,27791,Arena Men's Powerskin R-EVO Jammer Race,2,520.000000,199.679999
3,2021-06,27791,Arena Men's Powerskin R-EVO Jammer Race,2,520.000000,199.679999
4,2024-04,12950,camilla and marc Women's Expressionist Spot On...,2,460.000000,265.419999
5,2025-04,12736,Seafolly Women's Goddess Boyleg Maillot,3,424.800018,237.888010
6,2025-09,27821,Finis Men's Hydrospeed Velo Jammer Swimsuit,2,399.980011,161.191944
7,2023-04,28160,Quiksilver Men's Reactor Xplosive Boardshorts,2,398.000000,165.966000
8,2023-04,12929,Laundry by Shelli Segal Indian Summer One Piec...,3,384.000000,210.816000
9,2024-09,13081,Gottex Women's Chain Reaction Halter One Piece...,2,376.000000,219.959999


## Deep Dive into one Female Segment

##### Write a SQL query that calculates monthly gender-segment metrics — revenue, average order value (AOV), and repeat rate — using BigQuery syntax. Use the following tables: order_items with columns like order_id, product_id, and sale_price. products with columns like id, retail_price. orders with columns like order_id, user_id, gender, and order_date. The query should: Create a CTE (ip) to combine order_items with products, using COALESCE(sale_price, retail_price) as item_price. Create another CTE (gender_metrics) that: Groups results by gender and month (derived from order_date using FORMAT_DATE('%Y-%m', order_date)). Calculates: Unique buyers: COUNT(DISTINCT user_id) Revenue: SUM(item_price) AOV (Average Order Value): SUM(item_price) / NULLIF(COUNT(DISTINCT order_id), 0) Orders: COUNT(DISTINCT order_id) Repeat rate: fraction of users with more than one order in that month. Filter the data for gender = 'F'. Return results with columns: gender, month, revenue, AOV, repeat rate, and orders. Order results by month ascending. Use fully qualified table references like: mgmt-467-2500.Assignment_1.order_items mgmt-467-2500.Assignment_1.products mgmt-467-2500.Assignment_1.orders

In [ ]:
query_string = """
-- Gender segment revenue + AOV + repeat rate (monthly)
WITH ip AS (
  SELECT oi.order_id, COALESCE(oi.sale_price, p.retail_price) AS item_price, oi.product_id
  FROM `mgmt-467-2500.Assignment_1.order_items` oi
  LEFT JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
),
monthly_user_orders AS (
  SELECT
    user_id,
    FORMAT_DATE('%Y-%m', DATE(created_at)) AS month,
    COUNT(DISTINCT order_id) AS orders_per_user_month
  FROM `mgmt-467-2500.Assignment_1.orders`
  WHERE gender = 'F'
  GROUP BY 1, 2
),
gender_metrics AS (
  SELECT
    o.gender,
    FORMAT_DATE('%Y-%m', DATE(o.created_at)) AS month,
    COUNT(DISTINCT o.user_id) AS unique_buyers,
    SUM(ip.item_price) AS revenue,
    SUM(ip.item_price) / NULLIF(COUNT(DISTINCT o.order_id),0) AS aov,
    COUNT(DISTINCT o.order_id) AS orders,
    COUNT(DISTINCT CASE WHEN muo.orders_per_user_month > 1 THEN o.user_id END) / COUNT(DISTINCT o.user_id) AS repeat_rate
  FROM `mgmt-467-2500.Assignment_1.orders` o
  JOIN ip USING(order_id)
  JOIN monthly_user_orders muo ON o.user_id = muo.user_id AND FORMAT_DATE('%Y-%m', DATE(o.created_at)) = muo.month
  WHERE o.gender = 'F'
  GROUP BY 1, 2
)
SELECT * FROM gender_metrics
ORDER BY month ASC;
"""
query_job = client.query(query_string)

results_df = query_job.to_dataframe()
display(results_df)

# Validate

##### The earlier prompt to calculate Gross Profit Margin implied that you could directly calculate gross profit margin only by grouping monthly order data — but in some cases, gross profit margin should be derived from product-level or order-level data first, then aggregated.

In [ ]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE(o.created_at)) AS month,
  SUM(COALESCE(oi.sale_price, p.retail_price)) AS revenue,
  SUM(p.cost) AS cogs,
  SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost) AS gross_profit,
  SAFE_DIVIDE(SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost), SUM(COALESCE(oi.sale_price, p.retail_price))) AS gross_profit_margin
FROM `mgmt-467-2500.Assignment_1.order_items` oi
JOIN `mgmt-467-2500.Assignment_1.orders` o ON oi.order_id = o.order_id
JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
GROUP BY month
ORDER BY month ASC;
"""
query_job = client.query(query_string)

results_df = query_job.to_dataframe()
display(results_df)

##### Alternative query for "Swim" category deep dive

In [ ]:
query_string = """
SELECT
  FORMAT_DATE('%Y-%m', DATE(o.created_at)) AS month,
  p.id AS product_id,
  p.name,
  COUNT(1) AS units_sold,
  SUM(COALESCE(oi.sale_price, p.retail_price)) AS revenue,
  SUM(p.cost) AS total_cost,
  SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost) AS gross_profit,
  SAFE_DIVIDE(SUM(COALESCE(oi.sale_price, p.retail_price) - p.cost), SUM(COALESCE(oi.sale_price, p.retail_price))) AS gross_profit_margin
FROM `mgmt-467-2500.Assignment_1.order_items` oi
JOIN `mgmt-467-2500.Assignment_1.orders` o ON oi.order_id = o.order_id
JOIN `mgmt-467-2500.Assignment_1.products` p ON oi.product_id = p.id
WHERE p.category = 'Swim'
GROUP BY 1, p.id, p.name
ORDER BY month, revenue DESC;
"""
query_job = client.query(query_string)

results_df = query_job.to_dataframe()
display(results_df)

# Extend

#####
Create an appropriate plot to show the monthly revenue from earlier. dataframe is called results_df_mr

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.lineplot(data=results_df_mr, x='month', y='revenue')
plt.title('Monthly Revenue')
plt.xlabel('Month')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

1. Sustained Growth with Late Surge (Jan–Sep 2025):
Revenue steadily increased from around 280,000 dollars in January to over 750,000 dollars in September, suggesting effective business strategies such as improved marketing campaigns, expanded product lines, or stronger customer retention

Strategic Action: Investigate which initiatives during mid-year (e.g., June–August) drove acceleration scale or replicate those across other channels or regions


2. Sharp Decline in October 2025:
Revenue drops drastically in October to nearly $230K, erasing much of the growth trend. This could indicate seasonal demand, inventory shortages, or marketing spend reduction post-peak season

Strategic Action: Conduct root-cause analysis if seasonal, plan targeted promotions or early campaigns for Q4 to stabilize post-peak sales

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git config --global user.name "DanielGallagher1"
!git config --global user.email "gallagherdaniel555@gmail.com"

In [ ]:
from getpass import getpass
token = getpass('Enter your GitHub token: ')
!git clone https://{token}@github.com/garci843/Unit1_TheLook_Team1.git

In [ ]:
%cd Unit1_TheLook_Team1

In [ ]:
!cp "/content/drive/My Drive/MGMT467/Assignments/Unit1_Daniel_DIVE.ipynb" "/content/Unit1_TheLook_Team1/individual/"

In [ ]:
%cd /content/Unit1_TheLook_Team1
!git add individual/Unit1_Daniel_DIVE.ipynb
!git commit -m "Updated Unit1_Daniel_DIVE.ipynb with latest analysis"
!git push https://{token}@github.com/garci843/Unit1_TheLook_Team1.git main